# SK-8-MCP : Model Context Protocol et Integration

**Navigation** : [<< 07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | [Index](README.md)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre le **Model Context Protocol (MCP)**
2. Identifier les **serveurs MCP** disponibles
3. Integrer des **outils MCP** comme plugins SK
4. Creer un **bridge Agent Framework ↔ MCP**
5. Concevoir un **serveur MCP personnalise**

### Prerequis

- Python 3.10+
- Notebooks 01-07 completes
- Comprehension des Agents SK (notebook 03)
- Node.js installe (pour certains serveurs MCP)

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Qu'est-ce que MCP ? |
| 2 | Architecture | Tools, Resources, Prompts |
| 3 | Serveurs existants | Filesystem, GitHub, Database |
| 4 | SK + MCP | Integration comme plugins |
| 5 | Agent + MCP | Bridge avec Agent Framework |
| 6 | Serveur custom | Creer son propre serveur |
| 7 | Conclusion | Resume, exercices |

> **Model Context Protocol (MCP)** : Standard ouvert initie par Anthropic et adopte par Microsoft pour permettre aux LLMs d'acceder a des outils externes de maniere standardisee. C'est l'evolution des plugins vers l'interoperabilite.

In [1]:
# Installation
%pip install semantic-kernel python-dotenv mcp --quiet

import os
from dotenv import load_dotenv

load_dotenv()
print("MCP SDK installe")

Note: you may need to restart the kernel to use updated packages.
MCP SDK installe



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


## 1. Introduction au Model Context Protocol

### Pourquoi MCP ?

Avant MCP, chaque framework avait son propre systeme de plugins :

| Framework | Systeme de plugins | Probleme |
|-----------|-------------------|----------|
| Semantic Kernel | KernelFunction | Non portable |
| LangChain | Tools | Non portable |
| AutoGen | Tools | Non portable |
| OpenAI | Function Calling | Specifique OpenAI |

**MCP resout ce probleme** en definissant un standard :

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│ Semantic Kernel │     │   LangChain     │     │    AutoGen      │
│                 │     │                 │     │                 │
│  MCP Client     │     │  MCP Client     │     │  MCP Client     │
└────────┬────────┘     └────────┬────────┘     └────────┬────────┘
         │                       │                       │
         └───────────────────────┼───────────────────────┘
                                 │
                                 ↓
                    ┌─────────────────────────┐
                    │     MCP Servers         │
                    │                         │
                    │ - Filesystem            │
                    │ - GitHub                │
                    │ - Database              │
                    │ - Custom...             │
                    └─────────────────────────┘
```

### Acteurs principaux

| Acteur | Role | Exemple |
|--------|------|---------|
| **Client** | Consomme les outils | Claude Code, SK |
| **Server** | Fournit les outils | mcp-server-filesystem |
| **Transport** | Communication | stdio, HTTP, SSE |

## 2. Architecture MCP

MCP definit trois types de primitives :

### 2.1 Tools (Outils)

Fonctions executables par le LLM :

```json
{
  "name": "read_file",
  "description": "Read the contents of a file",
  "inputSchema": {
    "type": "object",
    "properties": {
      "path": { "type": "string" }
    },
    "required": ["path"]
  }
}
```

### 2.2 Resources (Ressources)

Donnees accessibles en lecture :

```json
{
  "uri": "file:///path/to/doc.md",
  "name": "Documentation",
  "mimeType": "text/markdown"
}
```

### 2.3 Prompts

Templates de prompts reutilisables :

```json
{
  "name": "summarize",
  "description": "Summarize a document",
  "arguments": [
    { "name": "content", "required": true }
  ]
}
```

### Tableau comparatif

| Primitive | Action | Equivalent SK |
|-----------|--------|---------------|
| **Tool** | Execute une fonction | KernelFunction |
| **Resource** | Lit des donnees | Pas d'equivalent direct |
| **Prompt** | Fournit un template | PromptTemplate |

## 3. Serveurs MCP existants

De nombreux serveurs MCP sont disponibles :

In [2]:
# Liste des serveurs MCP populaires
mcp_servers = [
    {
        "name": "filesystem",
        "package": "@anthropic/mcp-server-filesystem",
        "description": "Lecture/ecriture de fichiers",
        "tools": ["read_file", "write_file", "list_directory"]
    },
    {
        "name": "github",
        "package": "@anthropic/mcp-server-github",
        "description": "Operations GitHub (repos, issues, PRs)",
        "tools": ["search_repos", "get_issue", "create_pr"]
    },
    {
        "name": "postgres",
        "package": "@anthropic/mcp-server-postgres",
        "description": "Requetes SQL PostgreSQL",
        "tools": ["query", "execute", "describe_table"]
    },
    {
        "name": "jupyter",
        "package": "mcp-jupyter",
        "description": "Execution de notebooks Jupyter",
        "tools": ["execute_notebook", "read_cells", "manage_kernel"]
    },
    {
        "name": "puppeteer",
        "package": "@anthropic/mcp-server-puppeteer",
        "description": "Automatisation web (scraping, screenshots)",
        "tools": ["navigate", "screenshot", "click"]
    }
]

print("Serveurs MCP populaires:")
print("=" * 60)
for server in mcp_servers:
    print(f"\n{server['name'].upper()}")
    print(f"  Package: {server['package']}")
    print(f"  Description: {server['description']}")
    print(f"  Tools: {', '.join(server['tools'])}")

Serveurs MCP populaires:

FILESYSTEM
  Package: @anthropic/mcp-server-filesystem
  Description: Lecture/ecriture de fichiers
  Tools: read_file, write_file, list_directory

GITHUB
  Package: @anthropic/mcp-server-github
  Description: Operations GitHub (repos, issues, PRs)
  Tools: search_repos, get_issue, create_pr

POSTGRES
  Package: @anthropic/mcp-server-postgres
  Description: Requetes SQL PostgreSQL
  Tools: query, execute, describe_table

JUPYTER
  Package: mcp-jupyter
  Description: Execution de notebooks Jupyter
  Tools: execute_notebook, read_cells, manage_kernel

PUPPETEER
  Package: @anthropic/mcp-server-puppeteer
  Description: Automatisation web (scraping, screenshots)
  Tools: navigate, screenshot, click


### Installation d'un serveur MCP

```bash
# Via npm (Node.js requis)
npx -y @anthropic/mcp-server-filesystem --root /path/to/files

# Via uvx (Python)
uvx mcp-server-sqlite --db-path database.db
```

### Configuration dans Claude Code

Les serveurs MCP se configurent dans `~/.claude.json` :

```json
{
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": ["-y", "@anthropic/mcp-server-filesystem", "--root", "/path"]
    }
  }
}
```

## 4. Integration SK + MCP

Semantic Kernel peut consommer des serveurs MCP comme plugins.

In [3]:
# Exemple conceptuel : MCP Server comme plugin SK
# Note: L'integration native SK+MCP est en cours de developpement

from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from typing import Annotated
import subprocess
import json

class MCPFilesystemPlugin:
    """
    Wrapper qui expose un serveur MCP comme plugin SK.
    Pattern: MCP tools -> KernelFunctions
    """
    
    def __init__(self, root_path: str):
        self.root_path = root_path
    
    @kernel_function(description="Lit le contenu d'un fichier")
    def read_file(self, path: Annotated[str, "Chemin du fichier"]) -> str:
        """Wrapper pour l'outil MCP read_file."""
        full_path = os.path.join(self.root_path, path)
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            return f"Erreur: {e}"
    
    @kernel_function(description="Liste les fichiers d'un repertoire")
    def list_directory(self, path: Annotated[str, "Chemin du repertoire"] = ".") -> str:
        """Wrapper pour l'outil MCP list_directory."""
        full_path = os.path.join(self.root_path, path)
        try:
            files = os.listdir(full_path)
            return json.dumps(files, indent=2)
        except Exception as e:
            return f"Erreur: {e}"
    
    @kernel_function(description="Ecrit du contenu dans un fichier")
    def write_file(self, 
                   path: Annotated[str, "Chemin du fichier"],
                   content: Annotated[str, "Contenu a ecrire"]) -> str:
        """Wrapper pour l'outil MCP write_file."""
        full_path = os.path.join(self.root_path, path)
        try:
            with open(full_path, 'w', encoding='utf-8') as f:
                f.write(content)
            return f"Fichier ecrit: {full_path}"
        except Exception as e:
            return f"Erreur: {e}"

# Utilisation
kernel = Kernel()
kernel.add_plugin(MCPFilesystemPlugin(root_path="."), plugin_name="filesystem")

print("Plugin MCP Filesystem ajoute au Kernel")
print("\nFonctions disponibles:")
for func in kernel.get_plugin("filesystem").functions.values():
    print(f"  - {func.name}: {func.description}")

Plugin MCP Filesystem ajoute au Kernel

Fonctions disponibles:
  - list_directory: Liste les fichiers d'un repertoire
  - read_file: Lit le contenu d'un fichier
  - write_file: Ecrit du contenu dans un fichier


In [4]:
# Test du plugin
from semantic_kernel.functions import KernelArguments

# Lister les fichiers du repertoire courant
list_func = kernel.get_function("filesystem", "list_directory")
result = await kernel.invoke(list_func, KernelArguments(path="."))

print("Fichiers dans le repertoire courant:")
print(result)

Fichiers dans le repertoire courant:
[
  "01-SemanticKernel-Intro.ipynb",
  "02-SemanticKernel-Advanced.ipynb",
  "03-SemanticKernel-Agents.ipynb",
  "04-SemanticKernel-Filters-Observability.ipynb",
  "05-SemanticKernel-VectorStores.ipynb",
  "06-SemanticKernel-ProcessFramework.ipynb",
  "07-SemanticKernel-MultiModal.ipynb",
  "08-SemanticKernel-MCP.ipynb",
  "09-SemanticKernel-Building-CLR.ipynb",
  "10-SemanticKernel-NotebookMaker.ipynb",
  "10a-SemanticKernel-NotebookMaker-batch.ipynb",
  "10b-SemanticKernel-NotebookMaker-batch-parameterized.ipynb",
  "AutoGenNotebookUpdater.cs",
  "AutoInvokeSKAgentsNotebookUpdater.cs",
  "Cr\u00e9ateur de mail personnalis\u00e9.ipynb",
  "DisplayLogger.cs",
  "DisplayLoggerProvider.cs",
  "fort-boyard-csharp.ipynb",
  "fort-boyard-python.ipynb",
  "GuessingGame.cs",
  "MANIFEST.md",
  "Notebook-Generated.ipynb",
  "Notebook-Template.ipynb",
  "NotebookExecutor.cs",
  "NotebookPlannerUpdater.cs",
  "NotebookUpdaterBase.cs",
  "prompt_template_sampl

## 5. Bridge Agent Framework ↔ MCP

Integrons les outils MCP avec l'Agent Framework.

In [5]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory

# Configuration du kernel avec MCP plugin
kernel_with_mcp = Kernel()
kernel_with_mcp.add_service(OpenAIChatCompletion(service_id="agent"))
kernel_with_mcp.add_plugin(MCPFilesystemPlugin(root_path="."), plugin_name="filesystem")

# Configuration du function calling
settings = kernel_with_mcp.get_prompt_execution_settings_from_service_id("agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Creation de l'agent avec acces MCP
agent = ChatCompletionAgent(
    kernel=kernel_with_mcp,
    name="FileAssistant",
    instructions="""Tu es un assistant qui peut lire et manipuler des fichiers.
    Utilise les outils filesystem pour repondre aux demandes de l'utilisateur.
    Decris ce que tu fais avant chaque action.""",
    arguments=KernelArguments(settings=settings)
)

print("Agent avec outils MCP cree")
print(f"Outils disponibles: filesystem.read_file, filesystem.list_directory, filesystem.write_file")

Agent avec outils MCP cree
Outils disponibles: filesystem.read_file, filesystem.list_directory, filesystem.write_file


In [6]:
# Test de l'agent avec MCP
async def test_mcp_agent(query: str):
    """Teste l'agent avec une requete."""
    chat_history = ChatHistory()
    chat_history.add_user_message(query)
    
    print(f"User: {query}")
    print("-" * 40)
    
    async for content in agent.invoke(chat_history):
        if hasattr(content, 'content') and content.content:
            print(f"Agent: {content.content}")

# Test
try:
    await test_mcp_agent("Liste les fichiers Python dans le repertoire courant")
except Exception as e:
    print(f"Erreur: {e}")

User: Liste les fichiers Python dans le repertoire courant
----------------------------------------


Agent: Il n'y a pas de fichiers Python avec l'extension `.py` dans le répertoire courant. Toutefois, il y a plusieurs fichiers Jupyter Notebook avec l'extension `.ipynb` qui peuvent contenir du code Python. Voici une liste de ces fichiers :

- 01-SemanticKernel-Intro.ipynb
- 02-SemanticKernel-Advanced.ipynb
- 03-SemanticKernel-Agents.ipynb
- 04-SemanticKernel-Filters-Observability.ipynb
- 05-SemanticKernel-VectorStores.ipynb
- 06-SemanticKernel-ProcessFramework.ipynb
- 07-SemanticKernel-MultiModal.ipynb
- 08-SemanticKernel-MCP.ipynb
- 09-SemanticKernel-Building-CLR.ipynb
- 10-SemanticKernel-NotebookMaker.ipynb
- 10a-SemanticKernel-NotebookMaker-batch.ipynb
- 10b-SemanticKernel-NotebookMaker-batch-parameterized.ipynb
- Créateur de mail personnalisé.ipynb
- fort-boyard-python.ipynb
- Notebook-Generated.ipynb
- Notebook-Template.ipynb
- Semantic-kernel-AutoInteractive.ipynb
- Workbook-Template-Python.ipynb
- Workbook-Template.ipynb

Si tu veux explorer l'un de ces fichiers ou obtenir plus

### Interpretation : Agent + MCP

Ce pattern permet de creer des agents puissants :

```
┌─────────────────────────────────────────────────────┐
│                  Agent SK                           │
│  ┌─────────────────────────────────────────────┐   │
│  │              Kernel                          │   │
│  │  ┌───────────────┐  ┌───────────────────┐   │   │
│  │  │ MCP Plugins   │  │ Native Plugins    │   │   │
│  │  │ - filesystem  │  │ - custom logic    │   │   │
│  │  │ - github      │  │ - business rules  │   │   │
│  │  │ - database    │  │                   │   │   │
│  │  └───────────────┘  └───────────────────┘   │   │
│  └─────────────────────────────────────────────┘   │
│                         ↓                          │
│              FunctionChoiceBehavior.Auto()         │
│                         ↓                          │
│                    LLM Decision                    │
└─────────────────────────────────────────────────────┘
```

**Avantages** :
- Outils MCP standards reutilisables
- Plugins natifs pour la logique metier
- L'agent decide automatiquement quels outils utiliser

## 6. Creer un serveur MCP personnalise

Pour exposer vos propres outils via MCP.

In [7]:
# Exemple de serveur MCP en Python (conceptuel)
# Necessite le package 'mcp'

server_code = '''
from mcp.server import Server, NotificationOptions
from mcp.server.models import InitializationOptions
import mcp.server.stdio
import mcp.types as types

# Creation du serveur
server = Server("my-business-tools")

@server.list_tools()
async def handle_list_tools() -> list[types.Tool]:
    """Liste les outils disponibles."""
    return [
        types.Tool(
            name="calculate_price",
            description="Calcule le prix avec remise",
            inputSchema={
                "type": "object",
                "properties": {
                    "base_price": {"type": "number"},
                    "discount_percent": {"type": "number"}
                },
                "required": ["base_price"]
            }
        ),
        types.Tool(
            name="check_inventory",
            description="Verifie le stock d\'un produit",
            inputSchema={
                "type": "object",
                "properties": {
                    "product_id": {"type": "string"}
                },
                "required": ["product_id"]
            }
        )
    ]

@server.call_tool()
async def handle_call_tool(
    name: str, 
    arguments: dict
) -> list[types.TextContent]:
    """Execute un outil."""
    if name == "calculate_price":
        base = arguments["base_price"]
        discount = arguments.get("discount_percent", 0)
        final = base * (1 - discount/100)
        return [types.TextContent(type="text", text=f"Prix final: {final:.2f}")]
    
    elif name == "check_inventory":
        # Simulation de verification de stock
        product_id = arguments["product_id"]
        stock = 42  # Valeur simulee
        return [types.TextContent(type="text", text=f"Stock de {product_id}: {stock} unites")]
    
    return [types.TextContent(type="text", text="Outil non reconnu")]

async def main():
    async with mcp.server.stdio.stdio_server() as (read, write):
        await server.run(
            read, write,
            InitializationOptions(
                server_name="my-business-tools",
                server_version="1.0.0"
            )
        )

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
'''

print("Exemple de serveur MCP personnalise:")
print("=" * 60)
print(server_code)

Exemple de serveur MCP personnalise:

from mcp.server import Server, NotificationOptions
from mcp.server.models import InitializationOptions
import mcp.server.stdio
import mcp.types as types

# Creation du serveur
server = Server("my-business-tools")

@server.list_tools()
async def handle_list_tools() -> list[types.Tool]:
    """Liste les outils disponibles."""
    return [
        types.Tool(
            name="calculate_price",
            description="Calcule le prix avec remise",
            inputSchema={
                "type": "object",
                "properties": {
                    "base_price": {"type": "number"},
                    "discount_percent": {"type": "number"}
                },
                "required": ["base_price"]
            }
        ),
        types.Tool(
            name="check_inventory",
            description="Verifie le stock d'un produit",
            inputSchema={
                "type": "object",
                "properties": {
               

### Structure d'un serveur MCP

```
my-mcp-server/
├── pyproject.toml       # Configuration du package
├── src/
│   └── my_server/
│       ├── __init__.py
│       └── server.py    # Logique du serveur
└── README.md
```

### Configuration dans Claude Code

```json
{
  "mcpServers": {
    "my-business-tools": {
      "command": "python",
      "args": ["-m", "my_server.server"]
    }
  }
}
```

## 7. MCP dans notre infrastructure

Notre repository utilise deja MCP pour certaines operations.

### Serveur MCP Jupyter

Configure dans `notebook-infrastructure/mcp-maintenance/`, ce serveur permet :

| Outil | Description |
|-------|-------------|
| `execute_notebook` | Executer un notebook Papermill |
| `read_cells` | Lire les cellules d'un notebook |
| `manage_kernel` | Start/stop/restart kernels |
| `execute_on_kernel` | Executer du code sur un kernel actif |

### Integration avec NotebookMaker (05)

Le notebook [05-NotebookMaker](05-NotebookMaker.ipynb) pourrait etre etendu pour utiliser MCP :

```python
# Pattern actuel: Plugins locaux
@kernel_function
def update_cell(notebook_path: str, cell_index: int, content: str):
    # Logique locale
    pass

# Pattern futur: MCP
# L'agent appelle directement le serveur MCP Jupyter
# -> Plus de reutilisation, moins de code custom
```

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **MCP** | Standard d'interoperabilite | Protocol Client/Server |
| **Tool** | Fonction executable | `@server.call_tool()` |
| **Resource** | Donnee accessible | URI-based |
| **Prompt** | Template reutilisable | Arguments + Template |
| **SK Bridge** | MCP -> KernelFunction | Plugin wrapper |

## Points cles a retenir

1. **MCP = Standard ouvert** - Plugins portables entre frameworks
2. **Serveurs existants** - Filesystem, GitHub, Database, etc.
3. **SK integration** - MCP tools comme KernelFunctions
4. **Agent + MCP** - L'agent choisit les outils automatiquement
5. **Serveurs custom** - Exposez votre logique metier via MCP

## Exercices suggeres

1. **Wrapper GitHub** : Creer un plugin SK pour le serveur MCP GitHub
2. **Agent multi-MCP** : Agent utilisant filesystem + database
3. **Serveur metier** : Exposer une API interne via MCP

## Pour aller plus loin

| Ressource | Description |
|-----------|-------------|
| [MCP Specification](https://modelcontextprotocol.io/) | Documentation officielle |
| [MCP GitHub](https://github.com/anthropics/mcp) | SDK et serveurs |
| [SK MCP Integration](https://learn.microsoft.com/semantic-kernel) | Documentation SK |

---

**Navigation** : [<< 07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | [Index](README.md)

---

## Fin de la serie Semantic Kernel

Felicitations ! Vous avez complete les 8 notebooks de la serie :

| # | Notebook | Concepts |
|---|----------|----------|
| 01 | Fundamentals | Kernel, Services, Plugins, Chat |
| 02 | Functions | Function Calling moderne, Memory |
| 03 | Agents | ChatCompletionAgent, AgentGroupChat |
| 04 | Filters | Filtres, Logging, Observabilite |
| 05 | VectorStores | RAG, Qdrant, Embeddings |
| 06 | ProcessFramework | Workflows, Orchestration |
| 07 | MultiModal | DALL-E, Whisper, Vision |
| 08 | MCP | Interoperabilite, Serveurs |

**Prochaines etapes** :
- Explorer [05-NotebookMaker](05-NotebookMaker.ipynb) pour un exemple d'agents en production
- Consulter la [documentation SK](https://learn.microsoft.com/semantic-kernel)
- Suivre le [blog SK](https://devblogs.microsoft.com/semantic-kernel/) pour les nouveautes